# DART 기업 개황 수집

In [60]:
### 필요한 모듈
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
import xml.etree.ElementTree as ET
import pandas as pd

from time import sleep
import requests
import json


from datetime import datetime

In [2]:
api_key = "801b9fc87e380dfc40a19a593d6fd762bc681388"

In [56]:
corp_info_stock_code = pd.read_csv("corp_info_stock_code.csv", dtype = {'corp_code':str, 'stock_code':str})

In [65]:
corp_info_stock_code

,corp_code,corp_name,stock_code,modify_date
0,00260985,한빛네트,036720,20170630
1,00264529,엔플렉스,040130,20170630
2,00358545,동서정보기술,055000,20170630
3,00231567,애드모바일,032600,20170630
4,00247939,씨모스,037600,20170630
...,...,...,...,...
3560,00413417,우리손에프앤지,073560,20230403
3561,00440712,어반리튬,073570,20230403
3562,00483735,해성옵틱스,076610,20230403
3563,00516246,알에프세미,096610,20230403


In [39]:
corp_code_val = '00413417'

In [40]:
url = 'https://opendart.fss.or.kr/api/company.json?'
params = f'crtfc_key={api_key}&corp_code={corp_code_val}'

In [41]:
api_url = url + params

In [42]:
r = requests.get(api_url)

In [43]:
r_json = r.json()

In [44]:
df = pd.DataFrame(r_json, index  = [0])

In [46]:
total = pd.DataFrame()

In [ ]:
total = pd.concat([total, df])

In [49]:
test_for = dfdf.tail(10)

## for문을 활용하여 전체 기업 개황 데이터 수집

In [66]:
total = pd.DataFrame()
# for corp_code_val in test_for['corp_code']: # last 10 
for corp_code_val in corp_info_stock_code['corp_code']:  # total
    url = 'https://opendart.fss.or.kr/api/company.json?'
    params = f'crtfc_key={api_key}&corp_code={corp_code_val}'
    
    api_url = url + params    
    
    r = requests.get(api_url)
    r_json = r.json()
    df = pd.DataFrame(r_json, index  = [0])
    total = pd.concat([total, df])
    
    sleep(1)
    
    #print(api_url)
    print(corp_code_val)

00260985
00264529
00358545
00231567
00247939
00359614
00153551
00344746
00261188
00268020
00269287
00475286
00250775
00114321
00191108
00167378
00225210
00223346


KeyboardInterrupt: 

In [ ]:
total.to_csv('company_info.csv', index = False)

## 현재 시간

In [61]:
now = datetime.now()

In [63]:
now.time()

datetime.time(10, 22, 53, 33143)

In [64]:
now.strftime('%Y-%m-%d %H:%M:%S')

'2023-04-11 10:22:53'